In [2]:
in_size = list([3,4])
a = tuple([5]) 
b = tuple(in_size)
print(a)
print(b)

(5,)
(3, 4)


so I will have data as ndarray at bottleneck as N by D
I will have the class labels acquired by kmeans
I want to find correspondance samples as
a) two random samples that are in cluster
b) some sample and the center sample

which wil be acquired as in and out indices by a function that takes nxd array and cluster labels

In [8]:
import numpy as np
a = np.load('/home/doga/Desktop/correspondance_find.npz')
predictions = a['predictions']
bottleneck_vec = a['bottleneck_vec']

In [9]:
print(predictions.shape)
print(bottleneck_vec.shape)

(60000,)
(60000, 32)


In [19]:
uq_pr = np.unique(predictions)
for i in uq_pr:
    print("i=", int(i), ":", np.sum(predictions==i))

i= 0 : 1349
i= 1 : 2248
i= 2 : 2726
i= 3 : 3989
i= 4 : 1947
i= 5 : 857
i= 6 : 1224
i= 7 : 3478
i= 8 : 1196
i= 9 : 1340
i= 10 : 1268
i= 11 : 1692
i= 12 : 1810
i= 13 : 1256
i= 14 : 1659
i= 15 : 1791
i= 16 : 907
i= 17 : 2101
i= 18 : 1254
i= 19 : 1350
i= 20 : 1934
i= 21 : 2900
i= 22 : 1491
i= 23 : 1597
i= 24 : 1690
i= 25 : 2600
i= 26 : 2759
i= 27 : 2735
i= 28 : 1738
i= 29 : 2644
i= 30 : 641
i= 31 : 1829


In [21]:
import importlib as impL
import sys
sys.path.insert(1,'/home/doga/GithUBuntU/keyhandshapediscovery/vae_torch')
sys.path.insert(1,'/home/doga/GithUBuntU/keyhandshapediscovery')
import vae_torch_model as vtm
import helperFuncs as funcH

In [84]:
def get_correspondance_ids_01(X, cluster_ids):
    #only lists the cluster ids and number of samples in them
    uq_pr = np.unique(predictions)
    for i in uq_pr:
        cluster_id = i
        cluster_inds = funcH.getInds(predictions, i)
        # print("cluster_id=", int(cluster_id), ":", len(cluster_inds))
        
def get_correspondance_ids_02(X, cluster_ids):
    #uses X to find the center sample and returns center sample id too
    centroid_df = funcH.get_cluster_centroids(ft=X, predClusters=cluster_ids, verbose=0)
    uq_pr = np.unique(predictions)
    for i in range(len(uq_pr)):
        cluster_id = uq_pr[i]
        cluster_inds = funcH.getInds(predictions, i)
        #print("cluster_id({:-3d}), sampleCount({:-4d}), centerSampleId({:-5d})".format(int(cluster_id),len(cluster_inds),centroid_df['sampleID'].iloc[i]))

        
def get_correspondance_ids_03(X, cluster_ids):
    #uses X to find the center sample
    #returns inds_in, inds_out where:
    # inds_in : some_sample_id
    # inds_out: the center of cluster of that sample_id
    centroid_df = funcH.get_cluster_centroids(ft=X, predClusters=cluster_ids, verbose=0)
    uq_pr = np.unique(predictions)
    inds_in = []
    inds_out= []
    for i in range(len(uq_pr)):
        cluster_id = uq_pr[i]
        cluster_inds = funcH.getInds(predictions, i)
        center_sample_inds = centroid_df['sampleID'].iloc[i]
        print("cluster_id({:-3d}), sampleCount({:-4d}), centerSampleId({:-5d})".format(int(cluster_id),len(cluster_inds),center_sample_inds))
        #inds_in <--all sampleids except cluster center
        #inds_out<--cluster sample id with length of inds_in
        iin_cur = np.asarray(cluster_inds[np.where(center_sample_inds!=cluster_inds)],dtype=int).squeeze()
        out_cur = np.asarray(np.ones(iin_cur.shape)*center_sample_inds, dtype=int)
        print("iin_cur.shape{:}, out_cur.shape{:}".format(iin_cur.shape, out_cur.shape))
        if i==0:
            print("iin=", iin_cur)
            print("out=", out_cur)
        inds_in.append(iin_cur)
        inds_out.append(out_cur)  
    inds_in = np.asarray(np.concatenate(inds_in), dtype=int)
    inds_out = np.asarray(np.concatenate(inds_out), dtype=int)        
    return inds_in, inds_out

def get_cluster_correspondance_ids(X, cluster_ids, correspondance_type="shuffle"):
    #uses X to find the center sample
    #returns inds_in, inds_out where:
    #if correspondance_type=='shuffle'
    # inds_in : shuffled indices of a cluster
    # inds_out: shuffled indices of a cluster
    #elseif correspondance_type=='centered'
    # inds_in : some_sample_id
    # inds_out: the center of cluster of that sample_id
    centroid_df = funcH.get_cluster_centroids(ft=X, predClusters=cluster_ids, verbose=0)
    uq_pr = np.unique(predictions)
    inds_in = []
    inds_out= []
    for i in range(len(uq_pr)):
        cluster_id = uq_pr[i]
        cluster_inds = funcH.getInds(predictions, i)
        
        if correspondance_type=='shuffle':
            iin_cur = cluster_inds.copy()
            np.random.shuffle(iin_cur)
            out_cur = cluster_inds.copy()
            np.random.shuffle(out_cur)          
        else:
            center_sample_inds = centroid_df['sampleID'].iloc[i]
            print("cluster_id({:-3d}), sampleCount({:-4d}), centerSampleId({:-5d})".format(int(cluster_id),len(cluster_inds),center_sample_inds))
            #inds_in <--all sampleids except cluster center
            #inds_out<--cluster sample id with length of inds_in
            iin_cur = np.asarray(cluster_inds[np.where(center_sample_inds!=cluster_inds)],dtype=int).squeeze()
            out_cur = np.asarray(np.ones(iin_cur.shape)*center_sample_inds, dtype=int)
            
        print("iin_cur.shape{:}, out_cur.shape{:}".format(iin_cur.shape, out_cur.shape))
        if i==0:
            print("iin=", iin_cur)
            print("out=", out_cur)
        inds_in.append(iin_cur)
        inds_out.append(out_cur)
    inds_in = np.asarray(np.concatenate(inds_in), dtype=int)
    inds_out = np.asarray(np.concatenate(inds_out), dtype=int)        
    print("inds_in.shape{:}, inds_out.shape{:}".format(inds_in.shape, inds_out.shape))
    return inds_in, inds_out

In [85]:
inds_in, inds_out = get_cluster_correspondance_ids(bottleneck_vec, predictions, correspondance_type="shuffle")

iin_cur.shape(1349,), out_cur.shape(1349,)
iin= [49374 33561 45588 ... 31870 42364 56219]
out= [ 8245   955 47596 ... 21766 18661 42248]
iin_cur.shape(2248,), out_cur.shape(2248,)
iin_cur.shape(2726,), out_cur.shape(2726,)
iin_cur.shape(3989,), out_cur.shape(3989,)
iin_cur.shape(1947,), out_cur.shape(1947,)
iin_cur.shape(857,), out_cur.shape(857,)
iin_cur.shape(1224,), out_cur.shape(1224,)
iin_cur.shape(3478,), out_cur.shape(3478,)
iin_cur.shape(1196,), out_cur.shape(1196,)
iin_cur.shape(1340,), out_cur.shape(1340,)
iin_cur.shape(1268,), out_cur.shape(1268,)
iin_cur.shape(1692,), out_cur.shape(1692,)
iin_cur.shape(1810,), out_cur.shape(1810,)
iin_cur.shape(1256,), out_cur.shape(1256,)
iin_cur.shape(1659,), out_cur.shape(1659,)
iin_cur.shape(1791,), out_cur.shape(1791,)
iin_cur.shape(907,), out_cur.shape(907,)
iin_cur.shape(2101,), out_cur.shape(2101,)
iin_cur.shape(1254,), out_cur.shape(1254,)
iin_cur.shape(1350,), out_cur.shape(1350,)
iin_cur.shape(1934,), out_cur.shape(1934,)
iin_cur

In [70]:
print("inds_in.shape{:}, inds_out.shape{:}".format(inds_in.shape, inds_out.shape))

inds_in.shape(59968,), inds_out.shape(59968,)


In [79]:
a = np.arange(8)
print(a)
b = a.copy()
np.random.shuffle(b)
print(b)

[0 1 2 3 4 5 6 7]
[1 4 2 6 7 5 3 0]


In [104]:
def f(a, b, c):
    b.append(np.random.rand(2))    
    c.append(np.random.rand(2))
    return 5, 6
lab_vec = []
bottleneck_vec = []
f(5,lab_vec,bottleneck_vec)
z = f(5,lab_vec,bottleneck_vec)
print(lab_vec)
print(bottleneck_vec)
print(z[0])
print(z[1])

[array([0.8856084 , 0.83776979]), array([0.01579268, 0.47607407])]
[array([0.46853605, 0.68324035]), array([0.6675017 , 0.42673321])]
5
6


In [106]:
N=102
bs=13
fr=0
a = np.arange(N)
while fr<N:
    to = np.minimum(fr + bs, N)
    print("fr({:},to({:}),a{:}".format(fr,to,a[fr:to]))
    fr = to

fr(0,to(13),a[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
fr(13,to(26),a[13 14 15 16 17 18 19 20 21 22 23 24 25]
fr(26,to(39),a[26 27 28 29 30 31 32 33 34 35 36 37 38]
fr(39,to(52),a[39 40 41 42 43 44 45 46 47 48 49 50 51]
fr(52,to(65),a[52 53 54 55 56 57 58 59 60 61 62 63 64]
fr(65,to(78),a[65 66 67 68 69 70 71 72 73 74 75 76 77]
fr(78,to(91),a[78 79 80 81 82 83 84 85 86 87 88 89 90]
fr(91,to(102),a[ 91  92  93  94  95  96  97  98  99 100 101]
